# ***Initialization***

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# ***Importing Database***

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Rifat-87/Assignment-3-RNN/main/005930.KS.csv')
df.info()
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.fillna(method='ffill', inplace=True)
df1=df.reset_index()['Close']
plt.plot(df1)
plt.xlabel('Time')
plt.ylabel('Price')

In [ ]:
arima_model = ARIMA(df['Close'], order=(5, 1, 0))
arima_result = arima_model.fit()
df['ARIMA_Predictions'] = arima_result.predict(start=0, end=len(df) - 1, typ='levels')
df['Residuals'] = df['Close'] - df['ARIMA_Predictions']
scaler = MinMaxScaler(feature_range=(0, 1))
df['Normalized_Residuals'] = scaler.fit_transform(df[['Residuals']])
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back)]
        X.append(a)
        Y.append(dataset[i + look_back])
    return np.array(X), np.array(Y)

look_back = 5
X, Y = create_dataset(df['Normalized_Residuals'].values, look_back)


train_size = int(len(X) * 0.67)
test_size = len(X) - train_size
trainX, testX = X[0:train_size], X[train_size:]
trainY, testY = Y[0:train_size], Y[train_size:]


trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))


model = Sequential()
model.add(LSTM(50, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')


model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=1)

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)


trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)


predicted_residuals = np.append(trainPredict.flatten(), testPredict.flatten())
predicted_residuals = np.concatenate((np.zeros(look_back), predicted_residuals))
df['RNN_Predicted_Residuals'] = predicted_residuals[:len(df)]

df['Final_Predictions'] = df['ARIMA_Predictions'] + df['RNN_Predicted_Residuals']

plt.figure(figsize=(15, 8))
plt.plot(df['Close'], label='Actual Stock Price')
plt.plot(df['Final_Predictions'], label='Final Predicted Stock Price', alpha=0.7)
plt.title('Stock Price Prediction with Residual Correction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()
